# PyTorch & Plant Seedling Classification Tutorial

*Second Part*

In [8]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Data Extraction & Preprocessing

In [9]:
df = pd.read_csv('csv/data.csv', sep=';', index_col=0)

In [10]:
df.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p3063,p3064,p3065,p3066,p3067,p3068,p3069,p3070,p3071,label
0,65,56,60,68,58,61,64,53,59,68,...,96,77,62,95,76,61,103,83,65,Black-grass
1,53,40,36,50,37,34,50,40,39,54,...,47,40,41,50,41,42,49,42,44,Black-grass
2,148,152,154,165,176,183,172,178,186,172,...,59,34,24,71,54,43,78,60,48,Black-grass
3,100,86,66,85,65,46,87,73,51,84,...,91,70,42,101,70,41,99,70,43,Black-grass
4,66,49,35,98,82,61,129,114,88,133,...,105,91,70,84,70,54,79,63,50,Black-grass


---

In [11]:
x = df.drop('label', axis=1)
y = df.label

In [12]:
x.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p3062,p3063,p3064,p3065,p3066,p3067,p3068,p3069,p3070,p3071
0,65,56,60,68,58,61,64,53,59,68,...,66,96,77,62,95,76,61,103,83,65
1,53,40,36,50,37,34,50,40,39,54,...,41,47,40,41,50,41,42,49,42,44
2,148,152,154,165,176,183,172,178,186,172,...,20,59,34,24,71,54,43,78,60,48
3,100,86,66,85,65,46,87,73,51,84,...,48,91,70,42,101,70,41,99,70,43
4,66,49,35,98,82,61,129,114,88,133,...,73,105,91,70,84,70,54,79,63,50


In [13]:
y.head()

0    Black-grass
1    Black-grass
2    Black-grass
3    Black-grass
4    Black-grass
Name: label, dtype: object

---

In [14]:
l_encoder = LabelEncoder()
y = l_encoder.fit_transform(y)

In [15]:
x = np.array(x).reshape(-1, 32, 32, 3)
x = np.transpose(x, (0, 3, 1, 2))

---

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    random_state=42,
                                                    test_size=0.2)

---

In [17]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Net parameters

In [4]:
EPOCHES = 50
BATCH_SIZE = 32
NUM_CLASSES = 12

## Net Architecture

In [5]:
class MainNet(nn.Module):
    def __init__(self):
        super(MainNet, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout_1 = nn.Dropout2d(p=0.3)
        self.conv_5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.dense_1 = nn.Linear(3 * 3 * 512, 1024)
        self.dense_2 = nn.Linear(1024, 256)
        self.dense_3 = nn.Linear(256, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool_1(F.relu(self.conv_3(x)))
        x = self.pool_2(F.relu(self.conv_4(x)))
        x = F.relu(self.dropout_1(x))
        x = F.relu(self.conv_5(x))
        x = x.view(-1, 3 * 3 * 512)
        x = F.relu(self.dense_1(x))
        x = F.relu(self.dense_2(x))
        x = F.dropout(x, training=self.training)
        x = self.dense_3(x)
        return x

## Load Model

In [6]:
net = MainNet()
net.load_state_dict(torch.load('MainNet'))
net.eval()

MainNet(
  (conv_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_1): Dropout2d(p=0.3)
  (conv_5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (dense_1): Linear(in_features=4608, out_features=1024, bias=True)
  (dense_2): Linear(in_features=1024, out_features=256, bias=True)
  (dense_3): Linear(in_features=256, out_features=12, bias=True)
)

## Net Accuracy

In [20]:
def total_accuracy(net, x_test, y_test):
    correct = 0
    
    with torch.no_grad():
        for i in range(len(x_test)):
            logits = net(x_test[i: i + 1])
            _, pred = torch.max(logits, 1)
            if pred == y_test[i]:
                correct += 1

    print('TOTAL ACCURACY OF THE NET\n\t{0:.3f}'.format(correct / x_test.shape[0]))

In [18]:
def class_accuracy(net, x_test, y_test):
    correct = list(0 for i in range(NUM_CLASSES))
    total = list(0 for i in range(NUM_CLASSES))

    with torch.no_grad():
        for i in range(len(x_test)):
            logits = net(x_test[i: i + 1])
            _, pred = torch.max(logits, 1)
            mask = (pred == y_test[i])
            correct[y_test[i]] += mask.item()
            total[y_test[i]] += 1

    print('CLASSWISE ACCURACY\n')
    for i in range(NUM_CLASSES):
        print('\tACCURACY OF CLASS {0:2d}: {1:.3f}%'.format(i, 100 * correct[i] / total[i]))

---

In [23]:
total_accuracy(net, x_test, y_test)

TOTAL ACCURACY OF THE NET
	0.818


In [24]:
class_accuracy(net, x_test, y_test)

CLASSWISE ACCURACY

	ACCURACY OF CLASS  0: 27.397%
	ACCURACY OF CLASS  1: 90.000%
	ACCURACY OF CLASS  2: 75.385%
	ACCURACY OF CLASS  3: 95.364%
	ACCURACY OF CLASS  4: 58.974%
	ACCURACY OF CLASS  5: 87.379%
	ACCURACY OF CLASS  6: 88.028%
	ACCURACY OF CLASS  7: 67.442%
	ACCURACY OF CLASS  8: 88.288%
	ACCURACY OF CLASS  9: 74.242%
	ACCURACY OF CLASS 10: 93.103%
	ACCURACY OF CLASS 11: 82.105%
